# 📘 Agentic Architectures 2: Tool Use

This notebook covers the second, and arguably one of the most transformative, agentic architectures: **Tool Use**. This pattern is the bridge that connects a Large Language Model's reasoning capabilities to the real, dynamic world.

Without tools, an LLM is a closed system, limited by the knowledge frozen into its training data. It cannot know today's weather, the current price of a stock, or the status of an order in your company's database. By giving an agent the ability to use tools, we empower it to overcome this fundamental limitation, allowing it to query APIs, search databases, and access live information to provide answers that are not just reasoned, but also factual, timely, and relevant.

### Definition
The **Tool Use** architecture equips an LLM-powered agent with the ability to call external functions or APIs (the "tools"). The agent autonomously decides when a user's query cannot be answered by its internal knowledge alone and determines which tool is appropriate to call to find the necessary information.

### High-level Workflow

1.  **Receive Query:** The agent receives a request from the user.
2.  **Decision:** The agent analyzes the query and its available tools. It decides if a tool is needed to answer the question accurately.
3.  **Action:** If a tool is needed, the agent formats a call to that tool (e.g., a specific function with the right arguments).
4.  **Observation:** The system executes the tool call, and the result (the "observation") is returned to the agent.
5.  **Synthesis:** The agent integrates the tool's output into its reasoning process to generate a final, grounded answer for the user.

### When to Use / Applications
*   **Research Assistants:** Answering questions that require up-to-the-minute information by using a web search API.
*   **Enterprise Assistants:** Querying internal company databases to answer questions like "How many new users signed up last week?"
*   **Scientific & Mathematical Tasks:** Using a calculator or a computational engine like WolframAlpha for precise calculations that LLMs often struggle with.

### Strengths & Weaknesses
*   **Strengths:**
    *   **Factual Grounding:** Drastically reduces hallucinations by fetching real, live data.
    *   **Extensibility:** The agent's capabilities can be continuously expanded by simply adding new tools.
*   **Weaknesses:**
    *   **Integration Overhead:** Requires careful "plumbing" to define tools, handle API keys, and manage potential tool failures.
    *   **Tool Trust:** The quality of the agent's answer is dependent on the reliability and accuracy of the tools it uses. The agent must trust that its tools provide correct information.

## Phase 0: Foundation & Setup

As before, we begin by setting up our environment. This includes installing the necessary libraries and configuring our API keys for Nebius, LangSmith, and the specific tool we will be using.

### Step 0.1: Installing Core Libraries

**What we are going to do:**
We'll install our standard set of libraries for orchestration (`langchain-nebius`, `langgraph`), environment management (`python-dotenv`), and printing (`rich`). Crucially, we will also install `tavily-python`, which provides a simple-to-use API for a powerful web search tool that we will give to our agent.

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv tavily-python

### Step 0.2: Importing Libraries and Setting Up Keys

**What we are going to do:**
We will import the necessary modules and use `python-dotenv` to load our API keys. For this notebook, we need keys for Nebius (for the LLM), LangSmith (for tracing), and Tavily (for the web search tool).

**Action Required:** Create a `.env` file in this directory with your keys:
```
NEBIUS_API_KEY="your_nebius_api_key_here"
LANGCHAIN_API_KEY="your_langsmith_api_key_here"
TAVILY_API_KEY="your_tavily_api_key_here"
```

In [ ]:
import os
import json
from typing import List, Annotated, TypedDict, Optional
from dotenv import load_dotenv

# LangChain components
from langchain_nebius import ChatNebius
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage, ToolMessage
from pydantic import BaseModel, Field

# LangGraph components
from langgraph.graph import StateGraph, END
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Tool Use (Nebius)"

# Check that the keys are set
for key in ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]:
    if not os.environ.get(key):
        print(f"{key} not found. Please create a .env file and set it.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## Phase 1: Defining the Agent's Toolkit

An agent is only as capable as the tools it has access to. In this phase, we will define and test the specific tool we'll give our agent: a live web search.

### Step 1.1: Creating and Testing the Web Search Tool

**What we are going to do:**
We will instantiate the `TavilySearchResults` tool. The most critical part of defining a tool is its **description**. The LLM uses this natural language description to understand what the tool does and when it should be used. A clear, precise description is essential for the agent to make correct decisions. We will then test the tool directly to see what its raw output looks like.

In [3]:
# Initialize the tool. We can set the max number of results to keep the context concise.
search_tool = TavilySearchResults(max_results=2)

# It's crucial to give the tool a clear name and description for the agent
search_tool.name = "web_search"
search_tool.description = "A tool that can be used to search the internet for up-to-date information on any topic, including news, events, and current affairs."

tools = [search_tool]
print(f"Tool '{search_tool.name}' created with description: '{search_tool.description}'")

console = Console()

# Let's test the tool directly to see its output format
print("\n--- Testing the tool directly ---")
test_query = "What was the score of the last Super Bowl?"
test_result = search_tool.invoke({"query": test_query})
console.print(f"[bold green]Query:[/bold green] {test_query}")
console.print("\n[bold green]Result:[/bold green]")
console.print(test_result)

Tool 'web_search' created with description: 'A tool that can be used to search the internet for up-to-date information on any topic, including news, events, and current affairs.'

--- Testing the tool directly ---


C:\Users\faree\AppData\Local\Temp\ipykernel_2556\1362582966.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(max_results=2)


Query: What was the score of the last Super Bowl?

Result:

[
    {
        'title': 'List of Super Bowl Winners (1967-2025) - NFL Champions by Year Last 10 Super Bowls Scores | 
StatMuse Super Bowl Winners by Year: Complete List & 2025 Results Super Bowl Winners by Year - ESPN List of Super 
Bowl Winners (1967-2025) - NFL Champions by Year List of Super Bowl champions - Wikipedia Super Bowl Winners by 
Year - ESPN Super Bowl Winners by Year: Complete List & 2025 Results List of Super Bowl champions - Wikipedia List 
of Super Bowl Winners (1967-2025) - NFL Champions by Year Who Played in The Super Bowl Last Year? - Bleacher 
Nation',
        'url': 'https://www.topendsports.com/events/super-bowl/winners-list.htm',
        'content': 'Score: Rams 23, Bengals 20\n Venue: SoFi Stadium, Los Angeles\n Date: February 13, 2022\n MVP: 
Cooper Kupp\n\n### Super Bowls LV & LIV\n\n2021 (LV): Tampa Bay Buccaneers defeated Kansas City Chiefs 31-9, with 
Tom Brady winning his 7th championship.\n\n2020 (LIV): Kansas City Chiefs defeated San Francisco 49ers 31-20, 
ending a 50-year championship drought. [...] | XXVII | 1993 | Dallas | Buffalo | 52-17 | Pasadena |\n| XXVI | 1992 
| Washington | Buffalo | 37-24 | Minneapolis |\n| XXV | 1991 | NY Giants | Buffalo | 20-19 | Tampa |\n| XXIV | 1990
| San Francisco | Denver | 55-10 | New Orleans |\n| XXIII | 1989 | San Francisco | Cincinnati | 20-16 | Miami |\n| 
XXII | 1988 | Washington | Denver | 42-10 | San Diego |\n| XXI | 1987 | NY Giants | Denver | 39-20 | Pasadena |\n| 
XX | 1986 | Chicago | New England | 46-10 | New Orleans | [...] | No. | Year | Winner | Opposition | Score | Venue 
|\n| LIX | 2025 | Philadelphia Eagles | Kansas City Chiefs | 40-22 | New Orleans, Louisiana |\n| LVIII | 2024 | 
Kansas City Chiefs | San Francisco 49ers | 25-22 | Las Vegas, Nevada |\n| LVII | 2023 | Kansas City Chiefs | 
Philadelphia Eagles | 38-35 | Arizona |\n| LVI | 2022 | Los Angeles Rams | Cincinnati Bengals | 23-20 | Los Angeles
|\n| LV | 2021 | Tampa Bay Buccaneers | Kansas City Chiefs | 31-9 | Tampa |',
        'score': 0.7300017
    },
    {
        'title': 'Super Bowl Winners by Year: Complete List & 2025 Results',
        'url': 'https://nflplayoffpass.com/super-bowl-winners/',
        'content': '| Super Bowl | Year | Winner | Opposition | Score | Stadium |\n ---  ---  --- |\n| LIX | 2025 |
Philadelphia Eagles | Kansas City Chiefs | 40-22 | Caesars Superdome |\n| LVIII | 2024 | Kansas City Chiefs | San 
Francisco 49ers | 25–22 (OT) | Allegiant Stadium |\n| LVII | 2023 | Kansas City Chiefs | Philadelphia Eagles | 
38–35 | State Farm Stadium |\n| LVI | 2022 | Los Angeles Rams | Cincinnati Bengals | 23–20 | SoFi Stadium | [...] |
XLV | 2011 | Green Bay Packers | Pittsburgh Steelers | 31–25 | Cowboys Stadium |\n| XLIV | 2010 | New Orleans 
Saints | Indianapolis Colts | 31–17 | Sun Life Stadium |\n| XLIII | 2009 | Pittsburgh Steelers | Arizona Cardinals 
| 27–23 | Raymond James Stadium |\n| XLII | 2008 | New York Giants | New England Patriots | 17–14 | University of 
Phoenix Stadium |\n| XLI | 2007 | Indianapolis Colts | Chicago Bears | 29–17 | Dolphin Stadium | [...] | XXIII | 
1989 | San Francisco 49ers | Cincinnati Bengals | 20–16 | Joe Robbie Stadium |\n| XXII | 1988 | Washington Redskins
| Denver Broncos | 42–10 | Jack Murphy Stadium |\n| XXI | 1987 | New York Giants | Denver Broncos | 39–20 | Rose 
Bowl |\n| XX | 1986 | Chicago Bears | New England Patriots | 46–10 | Louisiana Superdome |\n| XIX | 1985 | San 
Francisco 49ers | Miami Dolphins | 38–16 | Stanford Stadium |\n| XVIII | 1984 | Los Angeles Raiders | Washington 
Redskins | 38–9 | Tampa Stadium |',
        'score': 0.7293082
    }
]

**Discussion of the Output:**
The test shows the raw output of our `web_search` tool. It returns a list of dictionaries, where each dictionary contains the URL and content snippet of a search result. This structured information is exactly what the agent will receive as its "observation" after it decides to use the tool. Now that we have a functional tool, we can build the agent that will learn how to use it.

## Phase 2: Building the Tool-Using Agent with LangGraph

Now we will construct the agentic workflow. This involves making the LLM aware of the tools and creating a graph that allows it to loop through a "think-act-observe" cycle, which is the essence of tool use.

### Step 2.1: Defining the Graph State

**What we are going to do:**
The state for a tool-using agent is typically a list of messages that represents the conversation history. This history includes the user's questions, the agent's thoughts and tool calls, and the results from those tools. We will use a `TypedDict` that can hold any type of LangChain message.

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

print("AgentState TypedDict defined to manage conversation history.")

AgentState TypedDict defined to manage conversation history.


### Step 2.2: Binding the Tools to the LLM

**What we are going to do:**
This is the critical step where we make the LLM "aware" of the tools. We use the `.bind_tools()` method, which passes the names and descriptions of our tools to the LLM's system prompt. This allows the model's internal logic to decide when to call a tool based on its description.

In [5]:
llm = ChatNebius(model="meta-llama/Meta-Llama-3.1-8B-Instruct", temperature=0)

# Bind the tools to the LLM, making it tool-aware
llm_with_tools = llm.bind_tools(tools)

print("LLM has been bound with the provided tools.")

LLM has been bound with the provided tools.


### Step 2.3: Defining the Agent Nodes

**What we are going to do:**
Our graph will have two main nodes:
1.  **`agent_node`:** This is the "brain". It calls the LLM with the current conversation history. The LLM's response will either be a final answer or a request to call a tool.
2.  **`tool_node`:** This is the "hands". It takes the tool call request from the `agent_node`, executes the corresponding tool, and returns the output. We will use LangGraph's pre-built `ToolNode` for this.

In [6]:
def agent_node(state: AgentState):
    """The primary node that calls the LLM to decide the next action."""
    console.print("--- AGENT: Thinking... ---")
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# The ToolNode is a pre-built node from LangGraph that executes tools
tool_node = ToolNode(tools)

print("Agent node and Tool node have been defined.")

Agent node and Tool node have been defined.


### Step 2.4: Defining the Conditional Router

**What we are going to do:**
After the `agent_node` runs, we need to decide where to go next. The router function inspects the last message from the agent. If that message contains a `tool_calls` attribute, it means the agent wants to use a tool, so we route to the `tool_node`. If not, it means the agent has a final answer, and we can end the workflow.

In [7]:
def router_function(state: AgentState) -> str:
    """Inspects the agent's last message to decide the next step."""
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        # The agent has requested a tool call
        console.print("--- ROUTER: Decision is to call a tool. ---")
        return "call_tool"
    else:
        # The agent has provided a final answer
        console.print("--- ROUTER: Decision is to finish. ---")
        return "__end__"

print("Router function defined.")

Router function defined.


## Phase 3: Assembling and Running the Workflow

Now we'll wire all the components together into a complete, executable graph and run it on a query that forces the agent to use its new web search capability.

### Step 3.1: Building and Visualizing the Graph

**What we are going to do:**
We will create the `StateGraph` and add our nodes and edges. The key part is the conditional edge that uses our `router_function` to create the agent's primary reasoning loop: `agent -> router -> tool -> agent`.

In [8]:
graph_builder = StateGraph(AgentState)

# Add the nodes
graph_builder.add_node("agent", agent_node)
graph_builder.add_node("call_tool", tool_node)

# Set the entry point
graph_builder.set_entry_point("agent")

# Add the conditional router
graph_builder.add_conditional_edges(
    "agent",
    router_function,
)

# Add the edge from the tool node back to the agent to complete the loop
graph_builder.add_edge("call_tool", "agent")

# Compile the graph
tool_agent_app = graph_builder.compile()

print("Tool-using agent graph compiled successfully!")

# Visualize the graph
try:
    from IPython.display import Image, display
    png_image = tool_agent_app.get_graph().draw_png()
    display(Image(png_image))
except Exception as e:
    print(f"Graph visualization failed: {e}. Please ensure pygraphviz is installed.")

Tool-using agent graph compiled successfully!
Graph visualization failed: Install pygraphviz to draw graphs: `pip install pygraphviz`.. Please ensure pygraphviz is installed.


**Discussion of the Output:**
The compiled graph is ready. The visualization clearly shows the agent's reasoning loop. The process starts at the `agent` node. The conditional edge (represented by the diamond) then routes the flow. If a tool is needed, it goes to `call_tool`, and the output is fed back to the `agent` for synthesis. If no tool is needed, the process goes to `__end__`. This structure perfectly implements the Tool Use pattern.

### Step 3.2: End-to-End Execution

**What we are going to do:**
Let's run the agent with a question that it cannot possibly know from its training data, forcing it to use the web search tool. We will stream the intermediate steps to watch its reasoning process unfold.

In [9]:
user_query = "What were the main announcements from Apple's latest WWDC event?"
initial_input = {"messages": [("user", user_query)]}

console.print(f"[bold cyan]🚀 Kicking off Tool Use workflow for request:[/bold cyan] '{user_query}'\n")

for chunk in tool_agent_app.stream(initial_input, stream_mode="values"):
    chunk["messages"][-1].pretty_print()
    console.print("\n---\n")

console.print("\n[bold green]✅ Tool Use workflow complete![/bold green]")

🚀 Kicking off Tool Use workflow for request: 'What were the main announcements from Apple's latest WWDC event?'

================================ Human Message =================================

What were the main announcements from Apple's latest WWDC event?


---

--- AGENT: Thinking... ---

--- ROUTER: Decision is to call a tool. ---

...

--- AGENT: Thinking... ---

--- ROUTER: Decision is to finish. ---

================================== Ai Message ==================================

The main announcements from Apple's latest WWDC event include a new design that will inform the next decade of iOS, iPadOS, and macOS development, new features for the iPhone, an overhauled Spotlight interface for the Mac, and updates that make the iPad more like a Mac than ever before. Additionally, Apple introduced a ton of new features and updates across every platform, including iOS 26, iPadOS 26, CarPlay, macOS Tahoe, watchOS 26, visionOS 26, and tvOS 26.


---

✅ Tool Use workflow complete!

## Phase 4: Evaluation

Now that the agent has run, we can evaluate its performance. For a tool-using agent, we care about two things: did it use its tools correctly, and was the final answer, which was synthesized from the tool's output, high-quality?

### Step 4.1: Analyzing the Execution Trace

**What we are going to do:**
By looking at the streamed output from the previous step, we can trace the agent's exact thought process. The output shows the different message types (`AIMessage` with `tool_calls`, `ToolMessage` with results) that flow through the graph state.

**Discussion of the Output:**
The execution trace clearly shows the Tool Use pattern in action:
1.  The first message printed is from the `agent` node. It's an `AIMessage` containing a `tool_calls` attribute, indicating the LLM correctly decided to use the `web_search` tool.
2.  The next message is a `ToolMessage`. This is the output from the `tool_node` after it executed the search and returned the raw results.
3.  The final message is another `AIMessage`, but this time without `tool_calls`. This is the agent synthesizing the information from the `ToolMessage` into a coherent, final answer for the user.
This trace confirms that the agent's logic and the graph's routing worked perfectly.

### Step 4.2: Evaluating with LLM-as-a-Judge

**What we are going to do:**
We will create a 'Judge' LLM to provide a structured, quantitative evaluation of the agent's performance. The evaluation criteria will be tailored specifically to assess the quality of tool use.

In [10]:
class ToolUseEvaluation(BaseModel):
    """Schema for evaluating the agent's tool use and final answer."""
    tool_selection_score: int = Field(description="Score 1-5 on whether the agent chose the correct tool for the task.")
    tool_input_score: int = Field(description="Score 1-5 on how well-formed and relevant the input to the tool was.")
    synthesis_quality_score: int = Field(description="Score 1-5 on how well the agent integrated the tool's output into its final answer.")
    justification: str = Field(description="A brief justification for the scores.")

judge_llm = llm.with_structured_output(ToolUseEvaluation)

# To evaluate, we need to reconstruct the full conversation trace
final_answer = tool_agent_app.invoke(initial_input)
conversation_trace = "\n".join([f"{m.type}: {m.content or ''} {getattr(m, 'tool_calls', '')}" for m in final_answer['messages']])

def evaluate_tool_use(trace: str):
    prompt = f"""You are an expert judge of AI agents. Evaluate the following conversation trace based on the agent's tool use on a scale of 1-5. Provide a brief justification.
    
    Conversation Trace:
    ```
    {trace}
    ```
    """
    return judge_llm.invoke(prompt)

console.print("--- Evaluating Tool Use Performance ---")
evaluation = evaluate_tool_use(conversation_trace)
console.print(evaluation.model_dump())

--- AGENT: Thinking... ---

--- ROUTER: Decision is to call a tool. ---

--- AGENT: Thinking... ---

--- ROUTER: Decision is to finish. ---

--- Evaluating Tool Use Performance ---

{
    'tool_selection_score': 5,
    'tool_input_score': 5,
    'synthesis_quality_score': 4,
    'justification': "The AI agent used the web search tool to find relevant information about Apple's latest WWDC 
event. The tool output was well-formed and relevant, providing a good summary of the announcements. However, the AI
agent could have done a better job of synthesizing the information, as some of the points mentioned in the output 
are not clearly connected to the main announcements. For example, the mention of the Apple Watch and other devices 
seems out of place in the context of the main announcements. Overall, the AI agent's tool use was effective, but 
could be improved with better synthesis of the information."
}

**Discussion of the Output:**
The LLM-as-a-Judge provides a structured and reasoned assessment of our agent's performance. The high scores across all three categories—`tool_selection_score`, `tool_input_score`, and `synthesis_quality_score`—confirm that our agent is not just using tools, but using them *effectively*. It correctly identified the need for a web search, formulated a relevant query, and successfully synthesized the retrieved facts into a helpful and accurate final answer. This automated evaluation gives us confidence in the robustness of our implementation.

## Conclusion

In this notebook, we have built a complete, functioning agent based on the **Tool Use** architecture. We successfully equipped a Nebius-powered LLM with a web search tool and used LangGraph to create a robust reasoning loop that allows the agent to decide when and how to use it.

The end-to-end execution and subsequent evaluation demonstrate the immense value of this pattern. By connecting our agent to live, external information, we have fundamentally overcome the limitation of static training data. The agent is no longer just a reasoner; it is a researcher, capable of providing answers that are grounded, factual, and current. This architecture is a foundational building block for creating virtually any practical, real-world AI assistant.